You can read the DICOM metadata - especially "Window Center", "Window Width", "Rescale Intercept" and "Rescale Slope" - for data pre-processing.
Head CT are a center of 40 and a width of 80

In [2]:
import sys
import os
from multiprocessing import Process
import concurrent.futures
import ast
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pydicom
import re
import PIL
from PIL import Image
from random import randrange
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns

train_csv_path = '../src/training.csv'
validate_csv_path = '../src/validation.csv'
test_csv_path = '../src/testing.csv'

tony_data_path = ''
chris_data_path = ''
kyle_data_path = '/media/keil/baltar/intracranial-hemorrhage-detection-data/stage_1_train_images/'

In [3]:
#dicoms to avoid
bad_actors = []

In [5]:
with open('bad_actors.csv', 'w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['id', 'cause'])
    for dicom in tqdm(os.listdir(kyle_data_path)):
        try:
            data = pydicom.dcmread(kyle_data_path+dicom)
        except:
            writer.writerow([dicom, 'corrupted'])
        img = np.array(data.pixel_array, dtype=float)
        if img.shape != (512, 512):
            print(dicom, img.shape)
            writer.writerow([dicom,img.shape])
    



# from multiprocessing.pool import ThreadPool as Pool

# pool_size = 5  
# # define worker function before a Pool is instantiated
# def worker(dicom):
#     try:
#         data = pydicom.dcmread(kyle_data_path+dicom)
#         img = np.array(data.pixel_array, dtype=float)
#     except:
#         writer.writerow([dicom, 'corrupted'])
    
#     if img.shape != (512, 512):
#         print(dicom, img.shape)
#         writer.writerow([dicom,img.shape])


# pool = Pool(pool_size)

# with open('bad_actors.csv', 'w') as csv_file:
#     writer = csv.writer(csv_file, delimiter=',')
#     writer.writerow(['id', 'cause'])
    
#     for dicom in tqdm(os.listdir(kyle_data_path)):
#         data = pool.apply_async(worker, (dicom,))

#     pool.close()
#     pool.join()

ID_dfa4e344a.dcm (638, 490)
ID_12e3b6923.dcm (430, 404)
ID_cef2af72d.dcm (638, 490)
ID_046ba342c.dcm (638, 490)
ID_680b2194c.dcm (666, 512)
ID_1bb3b44c7.dcm (638, 490)
ID_567a36143.dcm (436, 436)
ID_b4adf8739.dcm (462, 462)
ID_8d0ca7742.dcm (430, 404)
ID_db48a633d.dcm (666, 512)
ID_9cdc7295b.dcm (768, 768)
ID_c964e4096.dcm (666, 512)
ID_ca9462f49.dcm (462, 462)
ID_191369dca.dcm (408, 374)
ID_ac39010dc.dcm (430, 404)
ID_0e9ac1c5f.dcm (464, 464)
ID_d9840380c.dcm (464, 464)
ID_66131f4c9.dcm (408, 374)
ID_676b0cb59.dcm (768, 768)
ID_b76b13444.dcm (666, 512)
ID_898ff55b6.dcm (462, 462)
ID_37c495912.dcm (462, 462)
ID_a432727fd.dcm (464, 464)
ID_1bc5771a7.dcm (638, 490)
ID_69974dd3e.dcm (408, 374)
ID_3e31d57d0.dcm (666, 512)
ID_91c508c7a.dcm (462, 462)
ID_c35d5c858.dcm (430, 404)
ID_d6435f3bf.dcm (666, 512)
ID_b77ba3355.dcm (464, 464)
ID_a23a8193f.dcm (638, 490)
ID_9dad2eb09.dcm (666, 512)
ID_ea0ddbaf9.dcm (408, 374)
ID_d0c52575a.dcm (464, 464)
ID_c64131283.dcm (768, 768)
ID_f0d55b727.dcm (46

ValueError: The length of the pixel data in the dataset (153710 bytes) doesn't match the expected length (524288 bytes). The dataset may be corrupted or there may be an issue with the pixel data handler.

In [28]:
print(len(set(bad_actors)))

674258


In [4]:
# Helper Functions

def translate_dicom(filename, path=kyle_data_path, apply_window=True):
    """
    Transform a medical DICOM file to a standardized pixel based array
    Arguments:
        filename {string}
        path {string} -- file path to data, set in config.ini
        apply_window {bool} -- if True (default) then windowed png of dicom data is returned
    """
    
    data = pydicom.dcmread(path + filename)
    
    if apply_window:
        window_center, window_width, intercept, slope = get_windowing(data)
        img = window_image(data.pixel_array, window_center, window_width, intercept, slope)
        return np.array(img, dtype=float)

    img = np.array(data.pixel_array, dtype=float)
    standardized_array = np.divide(np.subtract(img,img.mean()),img.std())
    return standardized_array

def window_image(img, window_center, window_width, intercept, slope):
    """
    Given a CT scan img apply a windowing to the image
    Arguments:
        img {np.array} -- array of a dicom img processed by pydicom.dcmread()
        window_center,window_width,intercept,slope {floats} -- values provided by dicom file metadata
    """
    img = (img * slope + intercept)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max
    return img 

def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data.WindowCenter,
                    data.WindowWidth,
                    data.RescaleIntercept,
                    data.RescaleSlope]
    return (get_first_of_dicom_field_as_int(x) for x in dicom_fields)